# Day2
## 本章で学ぶ内容 
* 行動評価の指標となる**価値（累積報酬）の定義**
* 状態評価を**動的計画法で学習する手法と実装**

### 価値（累計報酬）を定式化するにあたっての課題
1. 将来の報酬が判明している必要がある  
&rarr; 再帰的に表現することで解決  
2. 将来の報酬の予測が、必ずあたるとしている  
&rarr; 期待値として見積もることで解決  

状態$s$において戦略$\pi$に基づいて行動した結果得られる価値を$V_\pi(s)$とし、  
現在の状態の価値を期待値で表す。  
  
$V_\pi(S_t)=E_\pi[r_{t+1}+\gamma V_\pi(S_{t+1})]$   
  
ここで、$r$は即時報酬、$\gamma$は割引率、$E[...]$は期待値を表す  
添え字の$t+1$は、行動後に得られる価値を表している

### Bellman Equationによる定義
期待値は、**行動確率**$\pi(a|s)$と**遷移確率**$T(s'|s,a)$を導入して次式で表すことができる  
$V_\pi(S_t)=\displaystyle\sum_a\pi(a|s)  
\displaystyle\sum_aT(s'|s,a)(R(s,s')+\gamma V_\pi(s'))$  
上記の式を<u>**Bellman Equation**</u>と呼ぶ  

**遷移確率**: 状態sの時に行動aをとった際に、状態s'に遷移する確率。  
**行動確率**: 状態sの時にaの行動をとる確率を戦略と呼び、特別に次式で表す  

参考になったページ  
https://gsnc.hatenablog.com/entry/2019/02/18/051713  
https://qiita.com/MENDY/items/77e9ae4cdd7c6476b080

### 2つの前提によるBellman Equationの単純化
1. 価値を最大とする行動をとる(Valueベース。戦略を基準とするPolicyベースもある)  
2. MDP（マルコフ決定課程）である(R(s,s')はR(s)となる)  
 
であるためBellman Equationは次式で表せる  
$V(s)=R(s)+\gamma max_a\displaystyle\sum_{s'}T(s'|s,a)V(s')$

## 実装編
### 環境
"up" か "down"の行動を5回行う。upだと＋1、downだと-1され、5回終了後に4以上だと成功。3以下は失敗。  
<div align="center">
<img src=stairs.png width=30%>
</div>

## 価値の定義
$V(s)=R(s)+\gamma max_a\displaystyle\sum_{s'}T(s'|s,a)V(s')$

In [41]:
def V(s, gamma=0.99):
    V = R(s) + gamma * max_V_on_next_state(s)
    return V

## 報酬関数
5回終了時に4以上で成功(happy end)か3以下で失敗(bad end)か。それ以外は0。
（5回終了時にしか報酬は無い）

In [42]:
def R(s):
    if s == "happy_end":
        return 1
    elif s == "bad_end":
        return -1
    else:
        return 0

## 価値の計算
Valueベースの実装  
$\gamma max_a\displaystyle\sum_{s'}T(s'|s,a)V(s')$

In [43]:
def max_V_on_next_state(s):
    # If game end, expected value is 0.
    if s in ["happy_end", "bad_end"]:
        return 0

    actions = ["up", "down"]
    values = []
    for a in actions:
        # 遷移確率の計算（次ページで定義）
        transition_probs = transit_func(s, a)
        v = 0
        for next_state in transition_probs:
            prob = transition_probs[next_state]
            # 遷移確率×遷移先の報酬
            v += prob * V(next_state)
        values.append(v)
    # Valueベースなので、最大の価値のみ返す
    return max(values)

## 遷移確率の計算

In [48]:
def transit_func(s, a):
     actions = s.split("_")[1:]
     LIMIT_GAME_COUNT = 5
     HAPPY_END_BORDER = 4
     # 選択した行動をとる確率を設定。
     MOVE_PROB = 0.9
 
     def next_state(state, action):
         return "_".join([state, action])
 
     if len(actions) == LIMIT_GAME_COUNT:
         up_count = sum([1 if a == "up" else 0 for a in actions])
         state = "happy_end" if up_count >= HAPPY_END_BORDER else "bad_end"
         prob = 1.0
         return {state: prob}
     else:
         opposite = "up" if a == "down" else "down"
         return {
             next_state(s, a): MOVE_PROB,
             next_state(s, opposite): 1 - MOVE_PROB
         }

 ## 確認
 指定の状態の価値を確認する
 

In [52]:
if __name__ == "__main__":
     print(V("state"))
     print(V("state_up_up_up_up_up"))
     print(V("state_down_down"))

0.7880942034605892
0.99
-0.96059601


 ## キーワード
 * Bellman Equation
   * ある状態における**価値（累計報酬）**の計算式
   * $V_\pi(S_t)=\displaystyle\sum_a\pi(a|s)\displaystyle\sum_aT(s'|s,a)(R(s,s')+\gamma V_\pi(s'))$
 * 動的計画法
   * 「モデルベース」の学習方法の一種
     * モデルベース:遷移関数、報酬関数をベースに学習するモデルのこと
     * モデルフリー:遷移関数、報酬関数が明確でない場合の手法。3章で学ぶ
   * 動的計画法の内、価値反復法について学ぶ
 * 行動決定の方法
   * Valueベース：**価値（累計報酬）**を最大化するように行動
   * Policyベース：戦略に基づいて行動

ここまでの実装は、  
取りうるすべての行動に対して価値を計算し、価値が最大となる行動  
をとるようにしたが、  
状態数が多い場合にすべての状態の計算を行うことは困難である。  

そこで・・・
<div align="center">
<img src=jouhou_hanran.png width=30%>
</div>

## 動的計画法（DP:Dynamic Programing）
ナップサック問題などに使われる、アルゴリズム界隈の言葉。  
以下、Wikipediaより引用

> https://ja.wikipedia.org/wiki/動的計画法  
> 細かくアルゴリズムが定義されているわけではなく、下記2条件を満たすアルゴリズムの総称である。  
> 1.分割統治法：部分問題を解き、その結果を利用して、問題全体を解く  
> 2.メモ化：部分問題の計算結果を再利用する  
>   
> 対象となる問題を帰納的に解く場合にくり返し出現する小さな問題例について、解を表に記録し表を埋めていく形で計算をすすめ、冗長な計算をはぶくアルゴリズムのことをいう。

今回は、動的計画法により各状態の価値を計算する**<u>価値反復法</u>（Value Iteration）**と、
戦略を学習する<u>**Policy Iteration**</u>を実装する

## 価値反復法（Value Iteration）
各状態の価値を、繰り返し計算を行うことで計算する。
基礎方程式は、Bellman equationを変形した以下の式  
$V_{i+1}(s)=max_a\displaystyle\sum_aT(s'|s,a)(R(s)+\gamma V_i(s'))$  
ここで、$i$は計算ステップを表しており、遷移先の価値は、ひとつ前の値（$V_i(s')$）を用いている。  
$V_i(s)$の値の更新が閾値を下回るまで繰り返し計算を行う。

<div align="center">
<img src=value.png width=50%>
</div>

In [46]:
class Planner():

    def __init__(self, env):
        self.env = env
        self.log = []

    def initialize(self):
        self.env.reset()
        self.log = []

    def plan(self, gamma=0.9, threshold=0.0001):
        raise Exception("Planner have to implements plan method.")
    # 遷移確率の関数の定義
    def transitions_at(self, state, action):
        # 状態、行動のペアから確率を計算
        transition_probs = self.env.transit_func(state, action)
        for next_state in transition_probs:
            prob = transition_probs[next_state]
            reward, _ = self.env.reward_func(next_state)
            yield prob, next_state, reward

    #状態の行列
    def dict_to_grid(self, state_reward_dict):
        grid = []
        for i in range(self.env.row_length):
            row = [0] * self.env.column_length
            grid.append(row)
        for s in state_reward_dict:
            grid[s.row][s.column] = state_reward_dict[s]

        return grid

In [47]:
#Plannerクラスを継承
class ValuteIterationPlanner(Planner):

    def __init__(self, env):
        super().__init__(env)

    def plan(self, gamma=0.9, threshold=0.0001):
        self.initialize()
        actions = self.env.actions
        V = {}
        for s in self.env.states:
            # Initialize each state's expected reward.
            # 最初の価値は、0とする
            V[s] = 0

        while True:
            delta = 0
            self.log.append(self.dict_to_grid(V))
            for s in V:
                if not self.env.can_action_at(s):
                    continue
                expected_rewards = []
                for a in actions:
                    r = 0
                    for prob, next_state, reward in self.transitions_at(s, a):
                        #Bellman Equationの価値反復法への適用
                        r += prob * (reward + gamma * V[next_state])
                    expected_rewards.append(r)
                #価値反復法なので、最大値のみを保持
                max_reward = max(expected_rewards)
                delta = max(delta, abs(max_reward - V[s]))
                V[s] = max_reward

            if delta < threshold:
                break

        V_grid = self.dict_to_grid(V)
        return V_grid

## Policy Iteration
**価値計算**、**戦略更新**の繰り返しを、戦略が更新されなくなるまで行う。
基礎方程式は、Bellman equationを変形した以下の式
$V_\pi(s)=\displaystyle\sum_a\pi(a|s)\displaystyle\sum_aT(s'|s,a)(R(s,s')+\gamma V_\pi(s'))$  
ここで、$i$は計算ステップを表しており、遷移先の価値は、ひとつ前の値（$V_i(s')$）を用いている。  
$V_i(s)$の値の更新が閾値を下回るまで繰り返し計算を行う。

<div align="center">
<img src=policy.png>
</div>